In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

In [0]:
import numpy as np
np.random.seed(7)

In [0]:
from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
input_size = np.prod(X_train.shape[1:])
print('input size -',input_size)

In [0]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [0]:
X_train = X_train.reshape((len(X_train), input_size))
X_test = X_test.reshape((len(X_test), input_size))

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

In [0]:
input_image = Input(shape=(input_size,), name='input_image')
embedding_size = 32

encoder_layer1 = Dense(128, activation='relu', name='encoder_layer1')(input_image)
encoder_layer2 = Dense(64, activation='relu', name='encoder_layer2')(encoder_layer1)

encoder_layer3 = Dense(embedding_size, activation='relu', name='encoder_layer3')(encoder_layer2)

decoder_layer1 = Dense(64, activation='relu', name='decoder_layer1')(encoder_layer3)
decoder_layer2 = Dense(128, activation='relu', name='decoder_layer2')(decoder_layer1)

decoder_layer3 = Dense(input_size, activation='sigmoid', name='decoder_layer3')(decoder_layer2)

autoencoder = Model(input_image, decoder_layer3)
autoencoder.summary()

In [0]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [0]:
autoencoder.fit(X_train, X_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

In [0]:
decoded_images = autoencoder.predict(X_test)

In [0]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_images[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()